# Import Dependencies

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import os
import requests
import json
api_key = os.getenv("API_KEY")

# Get data

In [ ]:
url = f"https://api.oikolab.com/weather"
response = requests.get(url,
    params={'param': 'temperature',
            'location': 'Toronto, Ontario',
            'start': '1990-01-01',
            'end': '2020-12-31'},
    headers={'api-key': api_key}
    )


In [ ]:
if response.status_code == 200:
    data = response.json()
    print('Success:', response.text)
else:
    print("Error:", response.status_code, response.text)

## Save Data

In [43]:
with open('react-demo/src/data/data.json', 'w') as f:
    json.dump(data, f)

## Load Data

In [16]:
with open('react-demo/src/data/data.json', 'r') as f:
    js = json.load(f)
    


In [17]:
data = json.loads(js['data'])


df = pd.DataFrame(index=pd.to_datetime(data['index'], unit='s'),
                  data=data['data'],
                  columns=data['columns'])

print(df.head())



                       coordinates (lat,lon) model (name)  \
1990-01-01 00:00:00  (43.653482, -79.383935)         era5   
1990-01-01 01:00:00  (43.653482, -79.383935)         era5   
1990-01-01 02:00:00  (43.653482, -79.383935)         era5   
1990-01-01 03:00:00  (43.653482, -79.383935)         era5   
1990-01-01 04:00:00  (43.653482, -79.383935)         era5   

                     model elevation (surface)  utc_offset (hrs)  \
1990-01-01 00:00:00                     127.19              -5.0   
1990-01-01 01:00:00                     127.19              -5.0   
1990-01-01 02:00:00                     127.19              -5.0   
1990-01-01 03:00:00                     127.19              -5.0   
1990-01-01 04:00:00                     127.19              -5.0   

                     temperature (degC)  
1990-01-01 00:00:00                2.37  
1990-01-01 01:00:00                2.07  
1990-01-01 02:00:00                1.82  
1990-01-01 03:00:00                1.13  
1990-01-01 04:

# Train Model

In [18]:
# Convert the temperature to fahrenheit
df['temperature (degF)'] = df['temperature (degC)'] * 9/5 + 32



In [19]:
scaler = MinMaxScaler(feature_range=(0, 1))
temperature_scaled = scaler.fit_transform(df[['temperature (degF)']].values)
window_size = 24 * 7  # Number of past days to use for predicting the next day's temperature

# Create sequences
def create_sequences(temperatures, window_size):
    X, y = [], []
    for i in range(len(temperatures) - window_size):
        X.append(temperatures[i:i + window_size])
        y.append(temperatures[i + window_size])
    return np.array(X), np.array(y).reshape(-1, 1)


# Use the normalized temperature data for creating sequences
X, y = create_sequences(temperature_scaled, window_size)

# Split the data into training and testing sets
# It's important not to shuffle time series data to maintain the temporal sequence
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Model Creation

In [20]:
# Adjusting the model if necessary
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X.shape[1], X.shape[2])),  # Increased complexity
    Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='mean_squared_error')

In [21]:
model.fit(X_train, y_train, epochs=1, validation_split=0.2)

5432/5432 [==============================] - 276s 51ms/step - loss: 0.0013 - val_loss: 1.2407e-04


# Model Prediction

In [22]:
latest_data = scaler.transform(df[['temperature (degF)']].values)[-window_size:].reshape((1, window_size, 1))
predicted_temperatures_normalized = []

# Recursive prediction
for _ in range(window_size):
    # Predict the next step
    next_step_normalized = model.predict(latest_data)
    
    # Store the normalized prediction
    predicted_temperatures_normalized.append(next_step_normalized[0, 0])
    
    # Update the input sequence with the new prediction
    # This moves the window one step forward by inserting the predicted value
    latest_data = np.roll(latest_data, -1, axis=1)
    latest_data[0, -1, 0] = next_step_normalized[0, 0]

# Convert normalized predictions back to the original scale (degrees Fahrenheit)
predicted_temperatures = scaler.inverse_transform(np.array(predicted_temperatures_normalized).reshape(-1, 1)).flatten()

print("Predicted temperatures for the next 7 days (168 hours):")
print(predicted_temperatures)


1/1 [==============================] - 0s 14ms/step
Predicted temperatures for the next 7 days (168 hours):
[33.93097  33.327564 32.896572 32.55162  32.264187 32.02234  31.823158
 31.673243 31.586046 31.583681 31.689096 31.921919 32.29525  32.811386
 33.46156  34.215588 35.00975  35.77763  36.462784 37.01894  37.422417
 37.669937 37.76939  37.73487  37.584057 37.341198 37.035355 36.695766
 36.35306  36.03853  35.76322  35.538395 35.376    35.291817 35.305035
 35.435326 35.698116 36.10141  36.63993  37.29564  38.012394 38.72288
 39.3697   39.904682 40.297073 40.53218  40.61405  40.55635  40.379322
 40.10659  39.7667   39.39037  39.007202 38.64794  38.32937  38.061848
 37.856903 37.729492 37.6991   37.7881   38.013744 38.382507 38.889793
 39.515682 40.20524  40.889946 41.51391  42.030567 42.405033 42.619213
 42.683216 42.611637 42.422565 42.138073 41.78571  41.394924 40.995975
 40.618645 40.283157 39.999332 39.77591  39.626163 39.569405 39.624775
 39.811333 40.14056  40.614326 41.223156 

In [24]:
model.save('react-demo/src/model/tempModel')
# model.save_weights()

2024-03-20 12:59:57.654206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,1]
	 [[{{node inputs}}]]
2024-03-20 12:59:57.658053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,1]
	 [[{{node inputs}}]]
2024-03-20 12:59:57.700166: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'lstm_1_input' with dtype float and shape [?,168,1]
	 [[{{node lstm_1_input}}]]
20

INFO:tensorflow:Assets written to: react-demo/src/model/tempModel/assets


INFO:tensorflow:Assets written to: react-demo/src/model/tempModel/assets


In [11]:
del model

In [12]:
load_model('demo/src/model/tempModel')